# LSA
    
토픽 모델링을 위해 최적화 된 알고리즘은 아니다.

BoW에 기반한 DTM, TF-IDF는 단어의 빈도수를 이용하기 떄문에 단어의 토픽을 고려하지 못한다.

이를 위해 DTM의 잠재된 의미를 이끌어내는 방법으로 잠재 의미 분석(LSA) 방법으로 극복한다.

선형대수학의 특이값 분해(SVD)의 원리를 사용한다.

(DTM: 다수의 문자 데이터에 등장한 모든 단어의 빈도수를 행렬로 표현한 것)

## 특이값 분해(Singular Calue Decomposition, Full SVD)

## 절단된 SVD

절단된 SVD는 대각 행렬의 값중 상위 t개만 남고 이를 통해 손실이 일어나 기존의 행렬 A를 복구할 수 없다.

또한 U,V행렬의 t열까지만 남기게 된다. 여기서 t는 우리가 찾고자 하는 토픽의 수를 반영한 하이퍼파라미터이다.

(t를 크게 잡으면 다양한 의미를 A로부터 뽑을 수 있지만, t를 작게 잡아야 노이즈 제거 가능하다)

t를 남겨 일부 벡터를 삭제하는 것을 데이터의 차원을 줄인다고 하는데, 이를 통해 계산 비용이 낮아지고 상대적으로 중요하지 않은 정보를 삭제 가능하다. 즉, 기존의 행렬에서 보지 못한 숨겨진 의미를 찾을 수 있다.

## 잠재 의미 분석(LSA)
LSA는 DTM이나 TF-IDF 행렬에 절단된 SVD를 사용해 차원을 축소시키고, 단어들의 잠재적 의미 이끌어낸다.

### Full SVD

In [1]:
import numpy as np

A = np.array([[0,0,0,1,0,1,1,0,0],[0,0,0,1,1,0,1,0,0],[0,1,1,0,2,0,0,0,0],[1,0,0,0,0,0,0,1,1]])
print('DTM의 크기(shape) :', np.shape(A))

DTM의 크기(shape) : (4, 9)


4x9의 크기를 가지는 DTM 생성 후 full SVD 수행해보자.

- S: 대각 행렬의 변수명
- VT: V의 전치행렬
- 소수점 둘째짜리까지만 출력한다.

In [2]:
U, s, VT = np.linalg.svd(A, full_matrices = True)
print('행렬 U :')
print(U.round(2))
print('행렬 U의 크기(shape) :',np.shape(U))

행렬 U :
[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]
행렬 U의 크기(shape) : (4, 4)


대각행렬 S 확인

In [5]:
print('특이값 벡터 :')
print(s.round(2))
print('특이값 벡터의 크기(shape) :',np.shape(s))

특이값 벡터 :
[2.69 2.05 1.73 0.77]
특이값 벡터의 크기(shape) : (4,)


In [6]:
#S를 다시 대각행렬로 변경해보자.

# 대각 행렬의 크기인 4 x 9의 임의의 행렬 생성
S = np.zeros((4, 9))

# 특이값을 대각행렬에 삽입
S[:4, :4] = np.diag(s)

print('대각 행렬 S :')
print(S.round(2))

print('대각 행렬의 크기(shape) :')
print(np.shape(S))

대각 행렬 S :
[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]
대각 행렬의 크기(shape) :
(4, 9)


위 대각행렬 S에서 값이 내림차순을 보인다.

In [7]:
print('직교행렬 VT :')
print(VT.round(2))

print('직교 행렬 VT의 크기(shape) :')
print(np.shape(VT))

직교행렬 VT :
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]
직교 행렬 VT의 크기(shape) :
(9, 9)


9x9의 직교행렬 생성했다.

In [8]:
np.allclose(A, np.dot(np.dot(U,S), VT).round(2))
#기존의 행렬 A와 UxSxVT 값은 같다.

True

### Truncated SVD
t = 2로 하고 Truncated SVD 수행해보자.

즉, S 내의 특이값 중 상위 2개만 남긴다.

In [9]:
# 특이값 상위 2개만 보존
S = S[:2,:2]

print('대각 행렬 S :')
print(S.round(2))

대각 행렬 S :
[[2.69 0.  ]
 [0.   2.05]]


In [11]:
#직교행렬 U에서도 2개의 열만 남기고 제거

U = U[:,:2]
print('행렬 U :')
print(U.round(2))

행렬 U :
[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


In [12]:
# VT도 마찬가지
VT = VT[:2,:]
print('직교행렬 VT :')
print(VT.round(2))

직교행렬 VT :
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


UxSxVT 연산을 해서 나오는 값을 A_prime이라 하고 A와 비교해보자.

A_prime은 정보의 손실이 일어나 완벽하게는 복구할 수 없을것이다.

In [13]:
A_prime = np.dot(np.dot(U,S), VT)
print(A)
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.  ]]


- 기존의 0인 값은 0에 가깝게 나오고 1인 값은 1에 가깝게 나온다.
- 축소된 U는 4x2의 값을 가지는데 이는 문서의 개수 x 토픽의 수 t이다. 이는 4개의 문서 각각을 2개의 값으로 표현한다고 할 수 있다.
- 즉, U의 각 행은 잠재 의미를 표현하기 위한 수치화 된 각각의 문서 벡터이다.
- 같은 의미로 VT의 각 열은 잠재 의미를 표현하기 위해 수치화된 각각의 단어 벡터라고 할 수 있다.

## 실습

Twenty Newsgroups의 20개 다른 주제를 가진 뉴스그룹 데이터에서 LSA를 활용해 문서의 수를 원하는 토픽의 수로 압축한 뒤 각 토픽당 가장 중요한 단어 5개를 출력해보자

### 데이터 불러오기

In [19]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import warnings
warnings.filterwarnings(action='ignore')


In [15]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))

샘플의 수 : 11314


In [16]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

위 뉴스그룹 데이터에서 원래 20개의 카테고리를 확인해보자.

In [17]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### 텍스트 전처리

알파벳을 제외한 문자 제거하고 짧은 단어 삭제, 그리고 알파벳을 소문자로 바꿔 단어의 개수 줄인다.

In [20]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [22]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

다음으로는 불용어 제거한다. 토큰화 후 불용어 제거.

In [24]:
# NLTK로부터 불용어를 받아온다.
import nltk
nltk.download('stopwords')

stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jihyeonbin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [26]:
# 토큰화 완료
print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


### TF-IDF 행렬 만들기

TF-IDF는 토큰화가 되어있지 않아야해 역토큰화 진행.

In [27]:
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [28]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

단어 1000개에 대한 TF-IDF 행렬을 TfidfVecotrizer를 통해 만들어보자.

In [29]:
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (11314, 1000)


### 토픽 모델링

TF-IDF 행렬을 Tuncated SVD를 통해 다수의 행렬로 분해 해보자. 20개의 토픽을 가졌다고 생각하고 모델링 시도한다.

토픽의 숫자: n_components

In [31]:
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

#svd_model.components_: LSA에서의 VT, V의 직교행렬을 의미

20

In [32]:
np.shape(svd_model.components_)

(20, 1000)

토픽의 수 x 단어의 수를 가진다.

In [33]:
#각 20개의 행의 각 1000개 열 중 가장 값이 큰 5개의 값 단어 찾아서 출력

terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('like', 0.21386), ('know', 0.20046), ('people', 0.19293), ('think', 0.17805), ('good', 0.15128)]
Topic 2: [('thanks', 0.32888), ('windows', 0.29088), ('card', 0.18069), ('drive', 0.17455), ('mail', 0.15111)]
Topic 3: [('game', 0.37064), ('team', 0.32443), ('year', 0.28154), ('games', 0.2537), ('season', 0.18419)]
Topic 4: [('drive', 0.53324), ('scsi', 0.20165), ('hard', 0.15628), ('disk', 0.15578), ('card', 0.13994)]
Topic 5: [('windows', 0.40399), ('file', 0.25436), ('window', 0.18044), ('files', 0.16078), ('program', 0.13894)]
Topic 6: [('chip', 0.16114), ('government', 0.16009), ('mail', 0.15625), ('space', 0.1507), ('information', 0.13562)]
Topic 7: [('like', 0.67086), ('bike', 0.14236), ('chip', 0.11169), ('know', 0.11139), ('sounds', 0.10371)]
Topic 8: [('card', 0.46633), ('video', 0.22137), ('sale', 0.21266), ('monitor', 0.15463), ('offer', 0.14643)]
Topic 9: [('know', 0.46047), ('card', 0.33605), ('chip', 0.17558), ('government', 0.1522), ('video', 0.14356)]
Topic 10